## Добавление индикаторов для нормализации

### 1. Стоимость фиксированного набора товаров и услуг

In [10]:
import sys
sys.path.insert(0, '..')

import pandas as pd
from reg_normalizer.regions_validator import RegionMatcher

In [11]:
data = pd.read_excel('../data/external/data_fixed_basket.xls')
data = data.rename(columns={'Стоимость фиксированного набора потребительских товаров и услуг (рубль)': 'indicator_value'})

In [12]:
data['indicator_value'] = data['indicator_value'].str.replace('входящий в состав Пермского края', '', regex=False)
data['indicator_value'] = data['indicator_value'].str.replace('входящий в состав Камчатского края', '', regex=False)
data['indicator_value'] = data['indicator_value'].str.replace('Агинский Бурятский округ (Забайкальский край)', 'Агинский Бурятский округ', regex=False)

data['indicator_value'] = data['indicator_value'].str.replace('Тюменская область (без АО)', 'Тюменская область без автономных округов', regex=False)
data['indicator_value'] = data['indicator_value'].str.replace('Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)', 'Тюменская область без автономных округов', regex=False)


data['indicator_value'] = data['indicator_value'].str.replace('Архангельская область (без АО)', 'Архангельская область без автономного округа', regex=False)
data['indicator_value'] = data['indicator_value'].str.replace('Архангельская область (кроме Ненецкого автономного округа)', 'Архангельская область без автономного округа', regex=False)

In [13]:
data = data.drop(index=0)
data = data.drop(index=2)
data.columns = data.iloc[0]
data = data.drop(index=1)
data = data.rename(columns={data.columns[0]: 'region'})
data.head()

1,region,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
3,Российская Федерация,3196.74,3577.03,4091.08,4709.74,5231.27,6068.25,6973.55,7714.12,8711.78,...,14212.11,14789.42,15467.92,16067.38,17052.67,18728.18,20224.9,NaN,NaN,NaN
4,Центральный федеральный округ,3329.09,3777.66,4339.65,5059.91,5616.82,6522.73,7502.09,8369.62,9470.95,...,15769.38,16678.92,17416.17,18537.87,19774.85,21603.35,22487.14,24275.42,26874.87,29474.36
5,Белгородская область,2747.37,3035.4,3538.49,4077.86,4544.95,5234.31,5954.31,6556.24,7286.67,...,12526.77,12933.25,13804.25,14110.96,14972.94,16022.72,17590.17,18438.06,20570.47,22338.71
6,Брянская область,2764.37,3037.41,3482.91,4006.58,4417.29,5185.99,5952.28,6685.62,7542.59,...,13293.24,13807.54,14504.38,14915.02,15612.58,17089.49,19294.32,20420.06,22168.65,23924.09
7,Владимирская область,2568.1,3005.14,3558.76,4125.27,4685.66,5683.05,6722.62,7257.76,8404.94,...,14056.93,14555.07,15331.3,15742.5,16586.44,17909.47,20134.67,21014.01,23939.4,26387.58


In [14]:
data['region'] = data['region'].str.lstrip()

In [15]:
matcher = RegionMatcher()

In [16]:
data = matcher.attach_fields(data, 'region', ['name_rus', 'oktmo'], threshold=0.5)

In [17]:
data.to_excel('../data/interim/data_fixed_basket_check.xlsx')

In [18]:
years = list(range(2000, 2026))

# Normalize year column names to strings so we can reliably melt.
rename_map = {}
for y in years:
    if y in data.columns:
        rename_map[y] = str(y)

temp_data = data.rename(columns=rename_map).copy()

for y in years:
    col = str(y)
    if col not in temp_data.columns:
        temp_data[col] = pd.NA

year_cols = [str(y) for y in years]

# Create new DataFrame for output
long_data = temp_data[['name_rus', 'oktmo', *year_cols]].melt(
    id_vars=['name_rus', 'oktmo'],
    value_vars=year_cols,
    var_name='year',
    value_name='indicator_value',
)

long_data['year'] = long_data['year'].astype(int)
# Sort data by name_rus and year (ascending)
long_data = long_data.sort_values(by=['name_rus', 'year']).reset_index(drop=True)

In [20]:
long_data

,name_rus,oktmo,year,indicator_value
0,Агинский Бурятский автономный округ,76800000,2000,<NA>
1,Агинский Бурятский автономный округ,76800000,2001,<NA>
2,Агинский Бурятский автономный округ,76800000,2002,2969.88
3,Агинский Бурятский автономный округ,76800000,2003,3296.65
4,Агинский Бурятский автономный округ,76800000,2004,3848.17
...,...,...,...,...
2777,Ярославская область,78000000,2021,17561.15
2778,Ярославская область,78000000,2022,19270.38
2779,Ярославская область,78000000,2023,19914.9
2780,Ярославская область,78000000,2024,22143.66


In [ ]:
#long_data.to_excel('../data/processed/fixed_basket_final.xlsx')

## 2. Индекс бюджетных расходов

In [21]:
ibr = pd.read_csv('../data/external/ibr_2005_2025.csv', sep=',')

In [22]:
ibr = ibr.rename(columns={'Субъекты РФ': 'region'})

In [23]:
ibr = matcher.attach_fields(ibr, 'region', ['name_rus', 'oktmo'], threshold=0.5)

In [24]:
ibr.to_excel('../data/interim/ibr_2005_2025_check.xlsx')

In [25]:
ibr

,region,2005,2006,2007,2008,2009,2010,2011,2012,2013,...,2018,2019,2020,2021,2022,2023,2024,2025,name_rus,oktmo
0,Белгородская область,"0,8073","0,7898","0,8206","0,8266","0,825","0,807","0,816","0,823","0,829",...,"0,838","0,842","0,847","0,847","0,848","0,848","0,848","0,831",Белгородская область,14000000
1,Брянская область,"0,8558","0,8407","0,8712","0,8756","0,877","0,857","0,87","0,877","0,869",...,"0,891","0,893","0,892","0,895","0,899","0,899","0,899","0,895",Брянская область,15000000
2,Владимирская область,"0,8123","0,7938","0,8548","0,8497","0,87","0,852","0,864","0,871","0,875",...,"0,891","0,894","0,896","0,893","0,892","0,892","0,892","0,906",Владимирская область,17000000
3,Воронежская область,"0,8201","0,8335","0,8431","0,8321","0,831","0,825","0,856","0,859","0,841",...,"0,835","0,836","0,838","0,837","0,84","0,84","0,84","0,838",Воронежская область,20000000
4,Ивановская область,"0,8515","0,8904","0,8617","0,8658","0,866","0,858","0,87","0,878","0,874",...,"0,883","0,89","0,889","0,882","0,877","0,877","0,877","0,835",Ивановская область,24000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,Амурская область,"1,641","1,5891","1,4473","1,451","1,401","1,395","1,417","1,411","1,426",...,"1,321","1,311","1,315","1,314","1,314","1,314","1,314","1,224",Амурская область,10000000
87,Магаданская область,"4,6349","4,3074","4,3293","4,725","4,804","4,824","4,698","4,639","4,575",...,"4,756","4,636","4,617","4,586","4,516","4,516","4,516","3,929",Магаданская область,44000000
88,Сахалинская область,"2,312","2,3005","2,2726","2,1897","2,183","2,138","2,128","2,112","2,14",...,"2,065","2,035","2,022","2,028","2,022","2,022","2,022","1,765",Сахалинская область,64000000
89,Еврейская автономная область,"1,5076","1,4931","1,3787","1,3257","1,3","1,298","1,299","1,29","1,329",...,"1,321","1,322","1,324","1,327","1,32","1,32","1,32","1,32",Еврейская автономная область,99000000


In [26]:
year_cols = [str(y) for y in range(2005, 2026)]

# Rename year columns to strings if they aren't already
rename_map = {y: str(y) for y in range(2005, 2026) if y in ibr.columns}
ibr_temp = ibr.rename(columns=rename_map).copy()

ibr_long = ibr_temp[['name_rus', 'oktmo', *year_cols]].melt(
    id_vars=['name_rus', 'oktmo'],
    value_vars=year_cols,
    var_name='year',
    value_name='indicator_value',
)

ibr_long['year'] = ibr_long['year'].astype(int)
ibr_long = ibr_long.sort_values(by=['name_rus', 'year']).reset_index(drop=True)
ibr_long

,name_rus,oktmo,year,indicator_value
0,Агинский Бурятский автономный округ,76800000,2005,"1,2173"
1,Агинский Бурятский автономный округ,76800000,2006,NaN
2,Агинский Бурятский автономный округ,76800000,2007,NaN
3,Агинский Бурятский автономный округ,76800000,2008,NaN
4,Агинский Бурятский автономный округ,76800000,2009,NaN
...,...,...,...,...
1906,Ярославская область,78000000,2021,"0,887"
1907,Ярославская область,78000000,2022,"0,886"
1908,Ярославская область,78000000,2023,"0,886"
1909,Ярославская область,78000000,2024,"0,886"


In [27]:
long_data = long_data.rename(columns={'indicator_value': 'fixed_basket_cost'})
ibr_long = ibr_long.rename(columns={'indicator_value': 'budget_expenditure_index'})

long_data = long_data.merge(ibr_long, on=['name_rus', 'oktmo', 'year'], how='outer')
long_data = long_data.sort_values(by=['name_rus', 'year']).reset_index(drop=True)
long_data

,name_rus,oktmo,year,fixed_basket_cost,budget_expenditure_index
0,Агинский Бурятский автономный округ,76800000,2000,<NA>,NaN
1,Агинский Бурятский автономный округ,76800000,2001,<NA>,NaN
2,Агинский Бурятский автономный округ,76800000,2002,2969.88,NaN
3,Агинский Бурятский автономный округ,76800000,2003,3296.65,NaN
4,Агинский Бурятский автономный округ,76800000,2004,3848.17,NaN
...,...,...,...,...,...
2798,Ярославская область,78000000,2021,17561.15,"0,887"
2799,Ярославская область,78000000,2022,19270.38,"0,886"
2800,Ярославская область,78000000,2023,19914.9,"0,886"
2801,Ярославская область,78000000,2024,22143.66,"0,886"


In [28]:
long_data.to_excel('../data/processed/economics_v20260211.xlsx')